# **Learning to Hack Weather Conditions**
#### A Machine Learning Workflow for Weather Forecasting
#### Team Twin AI ####

***Major Contributors to this Notebook:*** Zion Pibowei, Temitayo Adejuyigbe, Anosike Chimaobi Nice*

## Background

Formula 1 is one of the most competitive sports in the world. Engineers and technicians from every team use weather radar screens, provided by Ubimet to the teams, which allows them to track the current weather and make predictions during the race. Race engineers relay precise information to drivers, including:

- How many minutes until it starts raining
- Intensity of the rain
- Which corner will be hit first by the rain
- Duration of the rain

Points, and even races sometimes, are won and lost based on making sense of what the weather is going to do during a race, and being prepared as a team to act accordingly.

Therefore, weather forecasting takes a big part on the possible outcome of a race.

Similarly, F1 2021, the official Formula 1 videogame developed by Codemasters, uses a physics engine that behaves like the real world.

## The Challenge

In this challenge, we will analyse historical weather data from the RedBull Racing eSports team to build a high-performing model that is able to make accurate weather predictions/forecasts. Our objective is to predict the weather type 5, 10, 15, 30 and 60 minutes after a timestamp, and the rain percentage probability at that time. 

***Our solution is divided into 4 parts, each constituting a workflow on its own:***

- Part I: Initial Data Analysis and Preprocessing
- Part II: EDA and Feature Selection
- Part III: Modelling Methodology
- Part IV: Predictions and Exporting

## Part I: Initial Data Analysis and Preprocessing
<h4><b>Overview</b></h4>

This is the IDA and Preprocessing component of our solution to the FormulaAI Hack 2022 Competition. The workflow for this notebook is outlined as follows: 
- Getting the Data
- First Insights: Making Sense of the Data
- Data Integrity Assessments
- Cleaning the Data


In [ ]:
import os

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None, 'display.max_rows', 100)


import matplotlib.pyplot as plt
from matplotlib import figure
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('whitegrid')
sns.set_palette('Blues_r')

#!conda install deepchecks
import deepchecks as dc
from deepchecks.checks.integrity.is_single_value import IsSingleValue
from deepchecks.checks.integrity.data_duplicates import DataDuplicates
from deepchecks.checks import DataDuplicates
from deepchecks.checks.integrity import LabelAmbiguity
from deepchecks.base import Dataset, Suite

from sklearn.preprocessing import StandardScaler
from scipy import stats

import random
import time
from datetime import datetime

import warnings
# warnings.filterwarnings('ignore')


In [ ]:
#!conda install deepchecks -y

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Reading the CSV file
data = pd.read_csv('../input/formulaaihackathon2022/weather.csv',low_memory=False)
#data = pd.read_csv('weather.csv')

<br>
<h4><b>1. First Insights: Making Sense of the Data </b></h4>

In [ ]:
data.shape

In [ ]:
data.info() #check the overall information about the dataset

In [ ]:
data.head(2)

<br>In this dataframe, we see that the last column is an unnamed column. Before we drop it, let us inspect the proportion of NaN values it contains.<br>

In [ ]:
print('Missing values:',data['Unnamed: 58'].isnull().sum())
print('Proportion of missing values: {}%'.format(data['Unnamed: 58'].isnull().sum()/data.shape[0]*100))

Clearly, the unnamed column is entirely filled with missing values and, as such, has no impact in our workflow. It's presence in the data is most likely due to encoding. Thus, the first treatment to our data is to drop this column.

In [ ]:
data.drop('Unnamed: 58', axis = 1, inplace = True)
data.shape[1]

Now, let's obtain summary statistics for our data 

In [ ]:
data.describe()

<br><b>At a glance:</b>
- A quick inspection of the count row shows us that a number of columns contain missing values, ranging from small to large.
- A quick inspection of the standard deviation shows us that some columns have zero variance, indicating that <b>each of these columns contain ONLY ONE distinct value</b>. Typically, variables whose standard deviations tend to zero have fewer distinct values.
- A quick inspection of the min and max shows us that the very columns having 0 standard deviation <b>contain equal values of minimum and maximum</b>, validating our claim that these columns have only one distinct value.

In this project, we will carry out exhaustive analysis of the data to address the implictations of the forgoing discoveries.</b>


<h4><b>2. Data Integrity Assessments</b></h4>
<p>In this section, we will investigate the integrity of the data and uncover any data quality issues that may be present. The insights we obtain in this section will guide us on how to resolve these issues pragmatically in the next section.</p>

<p><b>(a) Unique Values</b></p>

Our ultimate goal is to build <b>a model that learns the evolution of weather conditions over time</b>. Therefore, we are interested in columns that show variation of values over time. Columns that contain only one unique value <b>may not provide predictive power for the model</b>. We will validate this assumption when we implement feature contribution checks ahead of our model methodology.
<p> First, we make a general inspection of the number of unique values contained in all the columns.</p>

In [ ]:
data.nunique(axis=0).sort_values().to_frame() #check for unique values and sort them into frames

From the above result, we can see that there are 7 columns that contain only one unique value. Below, we obtain further information about what these exact values are.

In [ ]:
sv = IsSingleValue()
sv.run(data)

<br>
<p><b>(b) Data Duplicates<b></p>
We need to run a duplicate check to find if there are multiple instances of identical samples in our dataset. One reason is that duplicates could be an indicator for a problem in the data pipeline that requires attention. The other is that they can potentially increase the weight that a machine learning model gives to samples. 

In [ ]:
print('Proportion of duplicates: {}%'.format(len(data[data.duplicated()])/data.shape[0]*100))
data[data.duplicated()] #check for average duplicate values in the dataset

Now, this is only partially informative. We only know that 2057230 samples, representing ~57% of the data, are duplicated. But this doesn't tell us the number of times each example of duplicate data appears. We will obtain the desired information by implementing the following additional checks.

In [ ]:
#from deepchecks.checks import DataDuplicates
DataDuplicates().run(data)

We can summarise this check by defining a check condition that sets the baseline of duplicate ratio as 0. This will expose any violation to the condition and reveal the present duplicate ratio.

In [ ]:
check = DataDuplicates()
check.add_condition_ratio_not_greater_than(0)
result = check.run(data)
result.show(show_additional_outputs=False)

We are interested in knowing whether the duplicates observed here were intentionally intended to be part of the data. However, if this is an hidden issue we’re not expecting to occur, then we will need to resolve it. We will revisit this in the EDA component of our workflow.

<br>
<p><b>(c) Label Ambiguity</b></p>

We would also like to check whether there are identical samples in the data with different labels. This alerts us to further verify whether or not the data was mislabelled, as mislabelled data could confuse the model and lead to lower model performance.

In [ ]:
label_ambig = Dataset(data, label='M_WEATHER')
LabelAmbiguity().run(label_ambig)

Again, we summarise this check by defining a check condition that sets the baseline of ambiguous sample ratio as 0. This will expose any violation to the condition and reveal the present ambiguous sample ratio.

In [ ]:
check = LabelAmbiguity()
check.add_condition_ambiguous_sample_ratio_not_greater_than(0)
result = check.run(label_ambig)
result.show(show_additional_outputs=False)

Indeed, we observe that there are no identical samples with different labels.

<br>
<p><b>(d) Missing Values</b></p>

In [ ]:
data.isna().sum().sort_values().to_frame() #check for missing values and sort them into frames

In [ ]:
data.notna().sum().sort_values().to_frame() #check for non-missing values and sort them into frames

From the above two cells, we immediately note the following:
- There are 18 columns with missing values, out of which 7 have only 1 missing value.
- Of the 18 columns, the number of missing values found in 8 columns (i.e., 974274 each) and the number found in 2 columns (i.e., 2598054) sum up to the length of the dataframe. 

Could there be a complimentary relationship, where columns in one set are filled in rows where those of the other set are missing?
To uncover this, we isolate the columns <b>M_WEATHER_PERCENTAGE and M_ZONE_START</b> and inspect the distribution of the missing vales across them. Due to the length of the dataframe, we slice a fraction of the data and visualise the distribution of missing values.

In [ ]:
xdf = data.copy()
xdf.M_RAIN_PERCENTAGE = np.where(xdf.M_RAIN_PERCENTAGE.isnull(),'1: Missing','1: Present')
xdf.M_ZONE_START = np.where(xdf.M_ZONE_START.isnull(),'2: Missing','2: Present')
xdf.M_ZONE_START.unique()

plt.figure(figsize=(20,30))
count = 0
for i in range(1,16):
    x1 = xdf[count:count+155].M_RAIN_PERCENTAGE
    x2 = xdf[count:count+155].M_ZONE_START
    index = range(count,count+155)
    plt.subplot(5,3,i)
    plt.plot(x1,index,'bo',markersize = 2,label='Rain percentage')
    plt.plot(x2,index,'ro',markersize = 2,label='Time zone start')
    plt.ylabel('Index') 
    plt.legend(loc='upper right')
    plt.grid(False)
    count+=155

From these plots, we can generalise that the missing values in one column appear in rows where the others are filled. This holds true for the other columns across the two sets. As the insights build up, we willgain better claarity on how to prepare the data to achieve overall completeness and accuracy.

<br>
<h4><b>4. Cleaning the Data</b></h4>

<b> (a) We will drop the following rows immediately </b>
1. Rows where the number of forcast samples equals 0 as they provide no prediction at time t = 0
2. Rows where the session type is unknown (0)
3. Rows where the packet received shows a session type of NaN or 0
4. Rows where the packet received is sent while the game is paused
5. Rows where the packet received shows player is both spectating and playing online (inconsistency)
6. Rows where marshal_zone_start or marshal_zone_flag is null, as these indicates gaps in the game

In [ ]:
df = data.copy()
df.shape

In [ ]:
df.drop(df[df['M_NUM_WEATHER_FORECAST_SAMPLES']==0].index, inplace=True)
df[df['M_NUM_WEATHER_FORECAST_SAMPLES']==0].count()

In [ ]:
df.shape

In [ ]:
df.drop(df[df['M_SESSION_TYPE']==0].index, inplace=True)
df.shape

In [ ]:
df.drop(df[df['M_GAME_PAUSED']==1].index, inplace=True)
df.shape

In [ ]:
df.drop(df[(df['M_IS_SPECTATING'] == 1) & (df['M_NETWORK_GAME'] == 1)].index, inplace=True)
df.shape

In [ ]:
df.drop(df[df['M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE'].isnull()].index, inplace=True)
df.shape

This last operation leaves the zone start and zone flag columns with NaN values as we can see below. We will therefore eliminate these columns in due time.

In [ ]:
df['M_ZONE_FLAG'].isnull().sum()

<br><b>(a) We will drop the following columns immediately:</b>
1. Redundant columns not included with the packet, starting with gamehost and timestamp
2. Redundant columns with single unique values (i.e., predominantly ID columns)
3. Duplicated Columns
4. Already Engineered Columns
5. Forcast samples columns outside weather and %rainfall, since this is a weather forecast project

We start by aggregating the session duration and Session time left column to generate a new column representing the time delta in the game.

In [ ]:
df['M_SESSION_TIME_SPENT'] = df['M_SESSION_DURATION'] - df['M_SESSION_TIME_LEFT']

In [ ]:
drop_col = ['GAMEHOST','TIMESTAMP', 'M_ZONE_FLAG', 'M_ZONE_START', 'M_SESSION_DURATION', 
           'M_SESSION_TIME_LEFT', 'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE', 
            'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE', 'M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE']

for col in df.columns:
    if df[col].nunique()<2:
        drop_col.append(col)
print(drop_col)
len(drop_col)

In [ ]:
df.drop(drop_col, axis=1, inplace=True)

In [ ]:
df.count()

In [ ]:
df.isna().any().sum()

<b>(c) Filling in the 6 columns containing single missing values</b>

In [ ]:
fill_col = df.columns[df.isna().any() == True]
for col in fill_col:
    df[col].fillna(df[col].mode()[0], inplace=True)
df.isna().any().sum()

In [ ]:
#Shape of data before cleaning
data.shape

In [ ]:
#Shape of data after cleaning
df.shape

In [1]:
#df.to_csv('cleaned_one.csv', encoding='utf-8', index=False)